In [1]:
!nvidia-smi

Mon Apr 14 17:18:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install accelerate
!pip install transformers
!pip install bitsandbytes
!pip install datasets
!pip install rouge-score
!pip install pymorphy2
!pip install peft
!pip install unsloth
#!pip install flash_attn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [1]:
import random
from typing import List, Dict

import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer
from tqdm import tqdm


class ChatDataset(Dataset):
    def __init__(
        self,
        original_records: List[Dict],
        tokenizer: AutoTokenizer,
        max_tokens_count: int,
        sample_rate: float = 1.0,
        only_target_loss: bool = True,
        add_global_bos: bool = True,
        add_global_eos: bool = True,
        labels_pad_token_id: int = -100,
    ):
        self.original_records = original_records
        self.sample_rate = sample_rate
        self.tokenizer = tokenizer
        self.max_tokens_count = max_tokens_count
        self.only_target_loss = only_target_loss
        self.labels_pad_token_id = labels_pad_token_id
        self.add_global_bos = add_global_bos
        self.add_global_eos = add_global_eos
        self.is_printed = False

        self.records = []
        for record in tqdm(original_records):
            if random.random() > self.sample_rate:
                continue
            tensors = self.convert_record(record)
            if tensors is None:
                continue
            self.records.append(tensors)

    def __len__(self):
        return len(self.records)

    def __getitem__(self, index):
        return self.records[index]

    def get_tokens(self, messages):
        #print(messages)
        tokens = self.tokenizer.apply_chat_template(
            messages,
            add_special_tokens=False,
            tokenize=True,
            add_generation_prompt=False,
        )
        if tokens[0] == self.tokenizer.bos_token_id:
            tokens = tokens[1:]
        return tokens

    def convert_record(self, record):
        input_ids, labels = [], []

        for i, message in enumerate(record["messages"]):
            if message['role'] == 'bot':
                message['role'] = 'assistant'
                record["messages"][i]['role'] = 'assistant'

            message_input_ids = self.get_tokens([message])
            message_labels = message_input_ids
            if len(input_ids) + len(message_input_ids) > self.max_tokens_count - 2:
                break

            labels_mask = [
                self.labels_pad_token_id for _ in range(len(message_input_ids))
            ]
            if (
                message["role"] not in ("assistant", "bot", "gpt")
                and self.only_target_loss
            ):
                message_labels = labels_mask

            input_ids.extend(message_input_ids)
            labels.extend(message_labels)

        if not input_ids:
            return None

        original_input_ids = self.get_tokens(record["messages"])
        if input_ids != original_input_ids[: len(input_ids)]:
            print(input_ids)
            print(original_input_ids[: len(input_ids)])
        assert input_ids == original_input_ids[: len(input_ids)]

        if self.add_global_bos and input_ids[0] != self.tokenizer.bos_token_id:
            input_ids.insert(0, self.tokenizer.bos_token_id)
            labels.insert(0, self.labels_pad_token_id)

        if input_ids[-2] == self.tokenizer.eos_token_id:
            input_ids = input_ids[:-1]
            labels = labels[:-1]

        if self.add_global_eos and input_ids[-1] != self.tokenizer.eos_token_id:
            input_ids.append(self.tokenizer.eos_token_id)
            labels.append(self.tokenizer.eos_token_id)

        if not self.is_printed:
            print(input_ids)
            print(labels)
            print(
                "Full prompt:" +
                self.tokenizer.decode(input_ids, skip_special_tokens=False)
            )
            assert '\n' in self.tokenizer.decode(input_ids, skip_special_tokens=False)
            self.is_printed = True

        input_ids = torch.LongTensor(input_ids)
        labels = torch.LongTensor(labels)
        attention_mask = input_ids.new_ones(input_ids.size())
        assert (
            input_ids.size(0)
            == labels.size(0)
            == attention_mask.size(0)
            <= self.max_tokens_count
        )
        return {
            "input_ids": input_ids,
            "labels": labels,
            "attention_mask": attention_mask,
        }

In [2]:
from datasets import load_dataset
dataset = load_dataset('IlyaGusev/saiga_scored')
dataset = dataset['train'].select(range(1000))
dataset = dataset.train_test_split(test_size=0.1)
dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/772 [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/115M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/41609 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['messages', 'source', 'opus_score', 'language', 'turns', 'sonnet_topic', 'sonnet_topic_explanation', 'sonnet_complexity', 'sonnet_complexity_explanation', 'is_bad_by_regex', 'score_explanation'],
        num_rows: 900
    })
    test: Dataset({
        features: ['messages', 'source', 'opus_score', 'language', 'turns', 'sonnet_topic', 'sonnet_topic_explanation', 'sonnet_complexity', 'sonnet_complexity_explanation', 'is_bad_by_regex', 'score_explanation'],
        num_rows: 100
    })
})

In [3]:
import random
import json
import os

import torch
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForTokenClassification,
    AutoConfig,
)
from transformers import (
    Trainer,
    TrainingArguments,
    logging,
    TrainerCallback,
    TrainerState,
    TrainerControl,
    BitsAndBytesConfig,
)
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
from unsloth import FastLanguageModel, UnslothTrainingArguments, UnslothTrainer

from peft import get_peft_model, LoraConfig
import re
from peft import prepare_model_for_kbit_training
import codecs

os.environ["WANDB_DISABLED"] = "true"

<ipython-input-3-ee3fbadfa80d>:23: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel, UnslothTrainingArguments, UnslothTrainer


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
instruct_model_name = 'Qwen/Qwen2.5-1.5B-Instruct'

tokenizer = AutoTokenizer.from_pretrained(instruct_model_name)
bos_token = tokenizer.bos_token
eos_token = tokenizer.eos_token
pad_token = tokenizer.pad_token

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [5]:
## чат темплейт у Qwen несколько странный в том плане, что он форсит системный промпт всегда, даже, когда мы этого не просим. Это осложняет некоторые процессы подготовки данных, когда требуется токенизация элементов диалога последовательно
## поэтому как вариант берем от RuadaptQwen2.5, который получен путем фикса этого случая (фикс не полный, с tool calling я не фиксил)
tokenizer = AutoTokenizer.from_pretrained('RefalMachine/RuadaptQwen2.5-7B-Lite-Beta')
chat_template = tokenizer.chat_template
chat_template

tokenizer_config.json:   0%|          | 0.00/11.0k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.34M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/2.28M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/12.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/759 [00:00<?, ?B/s]

'{%- if tools %}\n    {{- \'<|im_start|>system\\n\' }}\n    {%- if messages[0][\'role\'] == \'system\' %}\n        {{- messages[0][\'content\'] }}\n    {%- else %}\n        {{- \'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.\' }}\n    {%- endif %}\n    {{- "\\n\\n# Tools\\n\\nYou may call one or more functions to assist with the user query.\\n\\nYou are provided with function signatures within <tools></tools> XML tags:\\n<tools>" }}\n    {%- for tool in tools %}\n        {{- "\\n" }}\n        {{- tool | tojson }}\n    {%- endfor %}\n    {{- "\\n</tools>\\n\\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\\n<tool_call>\\n{\\"name\\": <function-name>, \\"arguments\\": <args-json-object>}\\n</tool_call><|im_end|>\\n" }}\n{%- else %}\n    {%- if messages[0][\'role\'] == \'system\' %}\n        {{- \'<|im_start|>system\\n\' + messages[0][\'content\'] + \'<|im_end|>\\n\' }}\n    {%- endif %}\n{%- en

In [6]:
model_name = 'Qwen/Qwen2.5-1.5B'
max_tokens_count = 1024
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_tokens_count,
    dtype=torch.float16,
    load_in_4bit=True,
    attn_implementation="sdpa",
)

==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.71k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [7]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536, padding_idx=151654)
    (layers): ModuleList(
      (0): Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
      (1-2):

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.bos_token = bos_token
tokenizer.eos_token = eos_token
tokenizer.pad_token = pad_token
tokenizer.chat_template = chat_template
tokenizer.padding_side = 'left'

tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [9]:
only_target_loss = True

datasets = []
for records in (dataset['train'], dataset['test']):
    datasets.append(
        ChatDataset(
            records,
            tokenizer,
            max_tokens_count=max_tokens_count,
            sample_rate=1.0,
            only_target_loss=only_target_loss,
            add_global_eos=False,
            add_global_bos=False
        )
    )
train_dataset, val_dataset = datasets

  2%|▏         | 21/900 [00:00<00:08, 105.94it/s]

[151644, 8948, 198, 33995, 4552, 1959, 30412, 283, 742, 31925, 13, 22419, 7972, 130036, 11073, 34789, 7819, 13103, 16964, 1959, 128632, 13932, 6442, 43758, 11, 127704, 10090, 32982, 43758, 7587, 20396, 6597, 125727, 32982, 43758, 5805, 7819, 12150, 49593, 131692, 23064, 127071, 39151, 13132, 134653, 16339, 38803, 125170, 134722, 133736, 13, 143956, 129171, 129760, 14062, 22787, 2247, 12228, 73626, 5805, 133304, 125635, 1504, 7587, 70522, 127073, 25428, 125439, 27016, 135193, 37557, 10090, 5805, 85895, 44022, 10474, 130599, 11, 129171, 129993, 127138, 20562, 128632, 133051, 125439, 27016, 135193, 37557, 10090, 11, 17686, 126478, 13695, 132890, 126231, 137908, 36998, 137028, 127001, 7587, 131704, 13932, 8005, 12228, 1802, 43758, 124686, 13, 136501, 33048, 131964, 130775, 129171, 61215, 9516, 9358, 5805, 14062, 1802, 132348, 13039, 14949, 1504, 127071, 39151, 13132, 11, 17307, 9358, 35397, 7587, 129856, 128805, 11310, 33393, 130102, 139984, 13, 56795, 18166, 271, 53645, 26991, 8178, 20200

 19%|█▉        | 19/100 [00:00<00:00, 180.27it/s]

[151644, 872, 198, 1792, 71705, 1478, 27016, 55757, 51329, 71490, 2184, 62984, 39151, 126621, 136789, 62129, 125286, 89605, 62133, 5805, 20396, 1792, 6442, 22787, 131049, 3780, 220, 151645, 198, 151644, 77091, 198, 16206, 140785, 78227, 1504, 132363, 27016, 55757, 51329, 141059, 71490, 2184, 62984, 39151, 126621, 136789, 11, 62129, 125286, 89605, 62133, 5805, 20396, 1792, 6442, 22787, 131049, 3780, 1447, 91, 96890, 220, 760, 128313, 16104, 5259, 126955, 136789, 1478, 664, 760, 42796, 6597, 37338, 129309, 62129, 40957, 6715, 5238, 7561, 9248, 91, 381, 91, 1408, 5161, 91, 3513, 9229, 12, 7360, 91, 220, 16, 220, 760, 58742, 15390, 3038, 14949, 13502, 3038, 34623, 24725, 18671, 320, 8281, 17, 8, 5180, 760, 128598, 10373, 22484, 56219, 5409, 126935, 38180, 10813, 28156, 11073, 93438, 320, 17753, 93766, 13764, 126227, 11, 12281, 12228, 128000, 60398, 42965, 53953, 4552, 11, 127623, 128968, 50636, 131228, 132857, 7336, 56086, 701, 125697, 22484, 56219, 38379, 127662, 286, 9248, 91, 220, 17, 2

100%|██████████| 100/100 [00:00<00:00, 159.18it/s]


In [10]:
messages = [{'role': 'user', 'content': 'Напиши что такое LLM.'}]
tokenizer.apply_chat_template(messages, return_tensors='pt', add_special_tokens=True, add_generation_prompt=True)

tensor([[151644,    872,    198,  20195,  19077,   1802,  29380,  47389, 134322,
            444,  10994,     13, 151645,    198, 151644,  77091,    198]])

In [11]:
from transformers import GenerationConfig

def generate(messages, model, tokenizer, generation_config):
    print(tokenizer.apply_chat_template(messages, add_special_tokens=False, add_generation_prompt=True, tokenize=False))
    input_ids = tokenizer.apply_chat_template(messages, return_tensors='pt', add_special_tokens=False, add_generation_prompt=True)
    input_ids = input_ids.to(model.device)
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            generation_config=generation_config
        )
    outputs = []
    for sample_output_ids, sample_input_ids in zip(output_ids, input_ids):
        sample_output_ids = sample_output_ids[len(sample_input_ids):]
        sample_output = tokenizer.decode(sample_output_ids, skip_special_tokens=True)
        outputs.append(sample_output)

    if len(outputs) == 1:
        outputs = outputs[0]
    return outputs



generation_config = GenerationConfig.from_dict(
    {
        'top_k': 40,
        'top_p': 0.9,
        'temperature': 0.2,
        'repetition_penalty': 1.0,
        'max_new_tokens': 64,
        'do_sample': True,
        'pad_token_id': tokenizer.pad_token_id,
        'bos_token_id': tokenizer.bos_token_id,
        'eos_token_id': tokenizer.eos_token_id
    }
)
generation_config

GenerationConfig {
  "do_sample": true,
  "eos_token_id": 151645,
  "max_new_tokens": 64,
  "pad_token_id": 151643,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}

In [12]:
generate(messages, model, tokenizer, generation_config)

<|im_start|>user
Напиши что такое LLM.<|im_end|>
<|im_start|>assistant



'桤ини\n桤ини\n桤ини\n桤ини\n桤ини\n桤ини\n桤ини\n桤ини\n桤ини\n桤ини\n桤ини\n桤ини\n桤ини\n桤ини\n桤ини\n桤ини\n'

In [13]:
lora_config = {
    "r": 16,
    "lora_alpha": 16,
    "lora_dropout": 0.0,
    "bias": "none",
    "target_modules": ["q_proj", "v_proj", "k_proj", "o_proj"],
    "use_gradient_checkpointing": "unsloth"
}

In [14]:
model = FastLanguageModel.get_peft_model(
    model, **lora_config, max_seq_length=max_tokens_count
)

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 0 MLP layers.


In [15]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536, padding_idx=151654)
        (layers): ModuleList(
          (0): Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
       

In [20]:
training_args = {
    "per_device_train_batch_size": 1,
    "per_device_eval_batch_size": 1,
    "gradient_accumulation_steps": 8,
    "eval_steps": 16,
    "save_steps": 128,
    "logging_steps": 16,
    "learning_rate": 0.00005,
    "num_train_epochs": 1,
    "lr_scheduler_type": "cosine",
    "warmup_steps": 16,
    "bf16": False,
    "fp16": True,
    "optim": "paged_adamw_8bit",
    "save_total_limit": 1,
    "seed": 1337,
    "max_grad_norm": 1.0,
    "weight_decay": 0.05
}
training_args = UnslothTrainingArguments(output_dir='./instruct_unsloth', **training_args)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [21]:
from unsloth.trainer import _create_unsloth_optimizer
class CustomTrainer(Trainer):
    def create_optimizer(self):
        embedding_learning_rate = getattr(self.args, "embedding_learning_rate", None)
        if embedding_learning_rate is None:
            return super().create_optimizer()
        if self.optimizer is None:
            optimizer_cls, optimizer_kwargs = Trainer.get_optimizer_cls_and_kwargs(self.args)
            self.optimizer = _create_unsloth_optimizer(
                self.model,
                optimizer_cls,
                optimizer_kwargs,
                embedding_learning_rate,
            )
        return self.optimizer

In [22]:
# Занимает в памяти всего 3.5GB для 1.5B модели при обучении!
trainer = data_collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=8)
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 888 | Num Epochs = 1 | Total steps = 111
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 4,358,144/5,000,000,000 (0.09% trained)


Step,Training Loss
16,1.371100
32,1.356200
48,1.385200
64,1.280100
80,1.209500
96,1.299300


TrainOutput(global_step=111, training_loss=1.3118418959883955, metrics={'train_runtime': 424.3667, 'train_samples_per_second': 2.093, 'train_steps_per_second': 0.262, 'total_flos': 3665863929716736.0, 'train_loss': 1.3118418959883955, 'epoch': 1.0})

In [23]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536, padding_idx=151654)
        (layers): ModuleList(
          (0): Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
       

In [24]:
model.save_pretrained('./trained_model_lora')

In [25]:
tokenizer.save_pretrained('./trained_model_lora')

('./trained_model_lora/tokenizer_config.json',
 './trained_model_lora/special_tokens_map.json',
 './trained_model_lora/vocab.json',
 './trained_model_lora/merges.txt',
 './trained_model_lora/added_tokens.json',
 './trained_model_lora/tokenizer.json')

In [26]:
!ls trained_model_lora

adapter_config.json	   merges.txt		    tokenizer_config.json
adapter_model.safetensors  README.md		    tokenizer.json
added_tokens.json	   special_tokens_map.json  vocab.json


In [27]:
generation_config

GenerationConfig {
  "do_sample": true,
  "eos_token_id": 151645,
  "max_new_tokens": 64,
  "pad_token_id": 151643,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}

In [28]:
generate(messages, model, tokenizer, generation_config)

<|im_start|>user
Напиши что такое LLM.<|im_end|>
<|im_start|>assistant



'LLM stands for Large Language Model. It refers to a type of artificial intelligence model that can understand and generate human-like text based on vast amounts of data. These models are designed to process and analyze large datasets, such as books, articles, and other written materials, to provide insights and responses to queries. LLM'